## Data Preprocessing 

In [95]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.float_format', '{:.2f}'.format)


In [96]:
# Read in the data as a DataFrame

file_path = 'data\Real_Estate_Sales_2001-2021_GL.csv'
df = pd.read_csv(file_path)


  df = pd.read_csv(file_path)


In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1054159 entries, 0 to 1054158
Data columns (total 14 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   Serial Number     1054159 non-null  int64  
 1   List Year         1054159 non-null  int64  
 2   Date Recorded     1054157 non-null  object 
 3   Town              1054159 non-null  object 
 4   Address           1054108 non-null  object 
 5   Assessed Value    1054159 non-null  float64
 6   Sale Amount       1054159 non-null  float64
 7   Sales Ratio       1054159 non-null  float64
 8   Property Type     671713 non-null   object 
 9   Residential Type  660275 non-null   object 
 10  Non Use Code      302242 non-null   object 
 11  Assessor Remarks  161472 non-null   object 
 12  OPM remarks       11564 non-null    object 
 13  Location          254643 non-null   object 
dtypes: float64(3), int64(2), object(9)
memory usage: 112.6+ MB


In [99]:
df.sample(5)

,Serial Number,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location
545560,80818,2008,09/29/2009,West Hartford,39 SUNRISE HILL DR,228200.00,320000.00,0.71,Single Family,Single Family,NaN,NaN,NaN,POINT (-72.75818 41.74491)
625058,100128,2010,01/31/2011,Torrington,270 ROOSEVELT AVE,129220.00,170000.00,0.76,Single Family,Single Family,NaN,NaN,NaN,NaN
558587,80335,2008,08/17/2009,Branford,81 WINDMILL HILL RD,165400.00,277000.00,0.60,Single Family,Single Family,NaN,NaN,NaN,NaN
544677,80056,2008,10/27/2008,Hartford,275 KENYON ST,154120.00,660000.00,0.23,Single Family,Single Family,NaN,NaN,NaN,NaN
690960,120093,2012,11/15/2012,Greenwich,"50 CONNECTICUT AVE, GREENWICH,",568750.00,905000.00,0.63,Single Family,Single Family,NaN,01-1885/s,NaN,NaN


In [100]:
# Check for missing values

df.isnull().sum()

Serial Number             0
List Year                 0
Date Recorded             2
Town                      0
Address                  51
Assessed Value            0
Sale Amount               0
Sales Ratio               0
Property Type        382446
Residential Type     393884
Non Use Code         751917
Assessor Remarks     892687
OPM remarks         1042595
Location             799516
dtype: int64

In [101]:
# Drop unecessary columns

df_new = df.drop(columns={'Assessor Remarks', 'Location', 'Address' , 'Date Recorded', 'Property Type'})

In [102]:
# Only get data on Single Family homes

df_sf = df_new[df_new['Residential Type'] == 'Single Family']

In [104]:
# Rename columns for easier processing

df_sf = df_sf.rename(columns = {'Serial Number' : 'serial_number', 'List Year' : 'list_year', 'Date Recorded' : 'date_recorded', 'Town' : 'town',
       'Assessed Value' : 'assessed_value','Sale Amount' : 'sale_amount', 'Sales Ratio' : 'sales_ratio', 'Property Type' : 'property_type', 'Residential Type' :
       'residential_type', 'Non Use Code' : 'non_use_code', 'OPM remarks': 'opm_remarks',
       })

Dataset description notes that the presence of a non usable sale code typically means the sale price is not reliable for use in the determination of a property value. Thus we will proceed to remove rows where non usable code values are present

In [105]:
# Drop where use non_use_code is NOT NaN because it indicates that the row is not valid for analysis

df_sf = df_sf[df_sf['non_use_code'].isna()]

Upon inspection we see that the OPM remarks notes if there was an error in reporting the sale or in the data entry process, thus we will examine that column for potential errors and remove the rows that show up after filtering.

In [109]:
# Make a copy out of DataFrame
df_opm = df_sf.copy()

# Drop rows where 'OPM Remarks' is missing 
df_opm = df_opm.dropna(subset=['opm_remarks'])

# List of keywords to filter out errors in entries
keywords = ['Incorrect', 'Wrong', 'Error', 'Faulty', 'Invalid', 'Mismatch', 'Inaccurate', 'Discrepancy', 'Typo', 'Misrecorded', 'Mistake', 'Incorrectly', 'Misentered']

# Filter out rows where 'OPM remarks' contains any of the keywords
df_opm_filtered = df_opm[df_opm['opm_remarks'].str.contains('|'.join(keywords), case=False, na=False)]

df_opm_filtered



,serial_number,list_year,town,assessed_value,sale_amount,sales_ratio,residential_type,non_use_code,opm_remarks
893345,170135,2017,Easton,762180.00,830000.00,0.92,Single Family,NaN,VALID SALE PER ASSESSOR APPEAL - CORRECTED CLE...


In [110]:
# Get the 'serial_number' of the rows to be dropped
serial_numbers_to_drop = df_opm_filtered['serial_number']

# Drop rows from df_sf where 'serial_number' is in the list of serial numbers to drop
df_sf = df_sf[~df_sf['serial_number'].isin(serial_numbers_to_drop)]

In [112]:
# Check for duplicates and drop them, but keep the latest occurence

df_sf = df_sf[~df_sf.duplicated(subset=['serial_number', 'assessed_value'], keep='last')]
df_sf = df_sf[~df_sf.duplicated(subset=['assessed_value', 'sale_amount'], keep='last')]

In [114]:
# Drop Unnecessary columns

df_sf = df_sf.drop(columns = {'residential_type', 'serial_number', 'opm_remarks', 'non_use_code'})

In [117]:
# Exclude properties that were sold and assessed at under 10K


df_sf = df_sf[df_sf['sale_amount'] > 10000.0]
df_sf = df_sf[df_sf['assessed_value'] > 10000.0]

In [118]:
# Inspect the outliers for sales_ratio, 

top_1_sales_ratio = df_sf['sales_ratio'].quantile(0.99)
top_1_ratio = df_sf[df_sf['sales_ratio'] >= top_1_sales_ratio]
top_1_ratio.sort_values(by = 'sales_ratio', ascending = False)

,list_year,town,assessed_value,sale_amount,sales_ratio
604195,2010,Hamden,110670208.00,208900.00,529.78
656891,2011,Chester,396440.00,45000.00,8.81
656020,2011,Chester,231950.00,27000.00,8.59
46567,2020,Norwich,161000.00,20000.00,8.05
571807,2009,North Haven,212310.00,26500.00,8.01
...,...,...,...,...,...
869693,2016,East Hampton,193650.00,175000.00,1.11
665687,2011,Stamford,343010.00,310000.00,1.11
654106,2011,Madison,531100.00,480000.00,1.11
989854,2019,Kent,243400.00,220000.00,1.11


In [119]:
# Drop the top 1% of sales_ratio 

top_1_percent_ratio = df_sf['sales_ratio'].quantile(0.99)
df_sf = df_sf[df_sf['sales_ratio'] <= top_1_percent_ratio]

In [121]:
df_sf.describe()

,list_year,assessed_value,sale_amount,sales_ratio
count,312016.00,312016.00,312016.00,312016.00
mean,2014.52,302264.07,468759.77,0.65
std,4.74,417913.94,651920.92,0.14
min,2006.00,10010.00,10300.00,0.04
25%,2011.00,135510.00,215000.00,0.56
50%,2015.00,194880.00,305000.00,0.65
75%,2019.00,307250.00,475000.00,0.74
max,2021.00,26609830.00,45000000.00,1.11


In [122]:
display(df_sf.sample(5))

,list_year,town,assessed_value,sale_amount,sales_ratio
992421,2018,Salisbury,531600.00,840000.00,0.63
508349,2007,East Haven,150220.00,198000.00,0.76
1029806,2019,Newington,119280.00,167000.00,0.71
16072,2020,Greenwich,300230.00,590000.00,0.51
994637,2019,Hamden,242480.00,283500.00,0.86


In [53]:
# Convert the cleaned DataFrame to a CSV file and download
df_sf.to_csv('cleaned_data.csv', index=False)